In [17]:
### physical level      体力活动水平   inactive;insufficiently active;active;highly active
### Sedentary_minutes  静坐时间-mins
import pandas as pd
import numpy as np

In [2]:
physical_activity_df = pd.read_sas('E:/work/Paper/NHANES/0910Analysis Data/Physical activity/PAQ_F.XPT')

In [3]:
# 从问卷中提取目标features
physical_activity_extract_feature = [
    'SEQN',
    'PAQ605',   #是否高强度 - 工作
    'PAQ610',   #高强度活动天数/周
    'PAD615',   #时间-分
    'PAQ620',   #是否中等强度 - 工作
    'PAQ625',   #中等强度活动天数/周
    'PAD630',   #时间-分
    'PAQ650',   #是否高强度 - 娱乐
    'PAQ655',   #天数
    'PAD660',   #分钟
    'PAQ665',   #是否中等强度 - 娱乐
    'PAQ670',   #天数
    'PAD675',   #分钟    
    'PAQ635',   #是否Walk or bicycle
    'PAQ640',   #天数
    'PAD645',   #分钟
    'PAD680',   #静坐时间
]


physical_activity_extract_df = physical_activity_df[physical_activity_extract_feature].fillna(0)
physical_activity_extract_df

,SEQN,PAQ605,PAQ610,PAD615,PAQ620,PAQ625,PAD630,PAQ650,PAQ655,PAD660,PAQ665,PAQ670,PAD675,PAQ635,PAQ640,PAD645,PAD680
0,51624.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,480.0
1,51625.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,51626.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,5.0,20.0,1.0,3.0,10.0,240.0
3,51627.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,51628.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,240.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9766,62155.0,2.0,0.0,0.0,1.0,6.0,120.0,2.0,0.0,0.0,1.0,4.0,180.0,2.0,0.0,0.0,240.0
9767,62156.0,2.0,0.0,0.0,1.0,1.0,30.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,600.0
9768,62157.0,1.0,4.0,300.0,1.0,3.0,180.0,2.0,0.0,0.0,1.0,4.0,240.0,1.0,3.0,45.0,360.0
9769,62158.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,480.0


In [4]:
# 
physical_minutes_and_level = pd.DataFrame()
physical_minutes_and_level['SEQN'] = physical_activity_extract_df['SEQN']

physical_minutes_and_level['moderate'] = (
    (physical_activity_extract_df['PAQ625'] * physical_activity_extract_df['PAD630']) + (physical_activity_extract_df['PAQ670'] * physical_activity_extract_df['PAD675'])
    + physical_activity_extract_df['PAQ640'] * physical_activity_extract_df['PAD645'] 
)
physical_minutes_and_level['vigorous'] = (
    (physical_activity_extract_df['PAQ610'] * physical_activity_extract_df['PAD615']) + (physical_activity_extract_df['PAQ655'] * physical_activity_extract_df['PAD660'])
)
physical_minutes_and_level['equal minutes'] = physical_minutes_and_level['moderate'] + 2*physical_minutes_and_level['vigorous']
physical_minutes_and_level['Sedentary minutes'] = physical_activity_extract_df['PAD680']

level_of_physical_activity = []
for i in physical_minutes_and_level['equal minutes'].to_list():
    if i == 0:
        level_of_physical_activity.append(0)
    elif i < 150:
        level_of_physical_activity.append(1)
    elif i <= 300:
        level_of_physical_activity.append(2)
    elif i > 300:
        level_of_physical_activity.append(3)
physical_minutes_and_level['physical_level'] = level_of_physical_activity

In [18]:
physical_activity_extract_df['PAD680'].replace([9999, 7777], np.nan, inplace=True)

In [19]:
physical_activity_extract_df['PAD680'].describe()

count    9771.000000
mean      260.977894
std       226.085632
min         0.000000
25%        40.000000
50%       240.000000
75%       420.000000
max      1200.000000
Name: PAD680, dtype: float64

In [20]:
physical_minutes_and_level['Sedentary_minutes'] = physical_activity_extract_df['PAD680']

In [21]:
physical_minutes_and_level.to_excel('E:/work/Paper/NHANES/0910Analysis Data/Final data/physical_level.xlsx')